Libraries

In [2]:
pip install fosforml

     |████████████████████████████████| 51kB 3.2MB/s eta 0:00:011
     |████████████████████████████████| 10.9MB 11.8MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 71.3MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 90.5MB/s eta 0:00:01
     |████████████████████████████████| 307kB 72.8MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 74.3MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 130kB/s s eta 0:00:01��███████▎              | 108.0MB 103.1MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 61.4MB/s eta 0:00:01     | 25.9MB 61.4MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 286kB/s  eta 0:00:01
     |████████████████████████████████| 61kB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 174kB 99.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 24.8MB/s eta 0:00:01
     |████████████████████████████████| 133kB 116.1MB/s eta 0:00:01
     |█████████████████████████████

In [3]:
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [5]:
sf_df = my_session.sql("select * from {}".format(table_name))
type(sf_df)
df=sf_df.to_pandas()
type(df)

pandas.core.frame.DataFrame

In [6]:
import pandas as pd

def generate_features(df):
    # Ensure the TRANS_DATE column is in datetime format
    df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
    
    # Extract temporal features
    df['year'] = df['TRANS_DATE'].dt.year
    df['month'] = df['TRANS_DATE'].dt.month
    df['day'] = df['TRANS_DATE'].dt.day
    df['dayofweek'] = df['TRANS_DATE'].dt.dayofweek
    df['quarter'] = df['TRANS_DATE'].dt.quarter
    df['is_month_start'] = df['TRANS_DATE'].dt.is_month_start
    df['is_month_end'] = df['TRANS_DATE'].dt.is_month_end
    
    # Create UNIT_PTR feature
    df['UNIT_PTR'] = df['SALES_PTR_VALUE'] / df['SALES_UNITS']
    
    return df

In [7]:
df = generate_features(df)

In [8]:
df = df.sort_values(by='TRANS_DATE')

In [9]:
# Convert all column names to uppercase
df.columns = [col.upper() for col in df.columns]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966400 entries, 867140 to 532214
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MNTH_CODE         966400 non-null  int32         
 1   TRANS_DATE        966400 non-null  datetime64[ns]
 2   START_DATE        966400 non-null  object        
 3   SALES_VALUE       966400 non-null  float64       
 4   SALES_UNITS       966400 non-null  int16         
 5   SALES_VOLUME      966400 non-null  float64       
 6   SALES_PTR_VALUE   966400 non-null  float64       
 7   OC_CODE           966400 non-null  int32         
 8   DISTRIBUTOR_CODE  966400 non-null  object        
 9   OUTLET_CODE       966400 non-null  object        
 10  CITY              966400 non-null  object        
 11  STATE             966400 non-null  object        
 12  COUNTY            966400 non-null  object        
 13  STREET            966400 non-null  object        
 14  PROD

In [11]:
df_sorted = df.sort_values(by=['OUTLET_CODE', 'PRODUCT_CODE', 'TRANS_DATE'])

In [12]:
pd.set_option('display.max_columns', None)

In [13]:
# List of columns in the desired order
columns_order = ['OUTLET_CODE', 'PRODUCT_CODE', 'TRANS_DATE', 'UNIT_PTR'] + [col for col in df_sorted.columns if col not in ['OUTLET_CODE', 'PRODUCT_CODE']]

# Reorder the DataFrame columns
df_sorted = df_sorted[columns_order]

In [14]:
df_sorted['FREQUENCY'] = df_sorted.groupby(['OUTLET_CODE', 'PRODUCT_CODE']).cumcount() + 1

K means clustering of OUTLET_CODES

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def label_encode_dataframe(df, columns_to_encode):
    # Initialize the label encoder
    label_encoder = LabelEncoder()
    
    # Apply label encoding to each column
    for column in columns_to_encode:
        df[column + '_encoded'] = label_encoder.fit_transform(df[column])
    
    # Convert all column names to uppercase and replace spaces with underscores
    df.columns = df.columns.str.upper().str.replace(' ', '_')
    
    return df

In [17]:
columns_to_encode = [ 'CATEGORY', 'SUBCATEGORY']
df_sorted = label_encode_dataframe(df_sorted, columns_to_encode)

In [18]:
df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966400 entries, 768449 to 8294
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   OUTLET_CODE          966400 non-null  object        
 1   PRODUCT_CODE         966400 non-null  object        
 2   TRANS_DATE           966400 non-null  datetime64[ns]
 3   UNIT_PTR             966400 non-null  float64       
 4   MNTH_CODE            966400 non-null  int32         
 5   TRANS_DATE           966400 non-null  datetime64[ns]
 6   START_DATE           966400 non-null  object        
 7   SALES_VALUE          966400 non-null  float64       
 8   SALES_UNITS          966400 non-null  int16         
 9   SALES_VOLUME         966400 non-null  float64       
 10  SALES_PTR_VALUE      966400 non-null  float64       
 11  OC_CODE              966400 non-null  int32         
 12  DISTRIBUTOR_CODE     966400 non-null  object        
 13  CITY            

In [26]:
# Select features for clustering
features = df_sorted[['SALES_UNITS', 'UNIT_PTR', 'FREQUENCY', 'CATEGORY_ENCODED', 'SUBCATEGORY_ENCODED']]

In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [28]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)  # You can choose the number of clusters
kmeans.fit(scaled_features)

KMeans(n_clusters=3)

In [29]:
df_sorted['Cluster'] = kmeans.labels_

In [35]:
df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966400 entries, 768449 to 8294
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   OUTLET_CODE          966400 non-null  object        
 1   PRODUCT_CODE         966400 non-null  object        
 2   TRANS_DATE           966400 non-null  datetime64[ns]
 3   UNIT_PTR             966400 non-null  float64       
 4   MNTH_CODE            966400 non-null  int32         
 5   TRANS_DATE           966400 non-null  datetime64[ns]
 6   START_DATE           966400 non-null  object        
 7   SALES_VALUE          966400 non-null  float64       
 8   SALES_UNITS          966400 non-null  int16         
 9   SALES_VOLUME         966400 non-null  float64       
 10  SALES_PTR_VALUE      966400 non-null  float64       
 11  OC_CODE              966400 non-null  int32         
 12  DISTRIBUTOR_CODE     966400 non-null  object        
 13  CITY            

In [34]:
# Convert all column names to uppercase
df_sorted.columns = [col.upper() for col in df_sorted.columns]

In [ ]:
df_sorted.to_csv('sales_clean_with_cluster.csv', index=False)